**annotationDeprecationMisuse.ipynb**

Misuse of properties intended for deprecated concepts only.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/annotationDeprecationMisuse.md

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [3]:
import sys
from rdflib import ConjunctiveGraph, Namespace

# Constants for script error reporting as per https://github.com/edamontology/edamverify.
NOERR = "NOERR"
INFO  = "INFO"
WARN  = "WARN"
ERROR = "ERROR"

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl', format='xml')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")



Loading graph ...

https://neuroml.org/neuromlv2|https://neuroml-db.org/ does not look like a valid URI, trying to serialize this will break.


done!


Define SPARQL query to retrieve ID, term, and attributes that are reservd for use on deprecated concepts. Run the query.

**NB.1:** BASE is used to define the define the default namespace (for various elements below).

**NB.2:** A concept is considered deprecated if it has a property ``<oboInOwl:inSubset>`` whose value contains the string ``obsolete``, e.g. ``<oboInOwl:inSubset rdf:resource="http://purl.obolibrary.org/obo/edam#obsolete"/>`` or ``<oboInOwl:inSubset>obsolete/>``.

In [4]:
# Compile SPARQL query
query_term = """
BASE <http://edamontology.org/>
SELECT ?id ?term ?obsolete_since ?subsetpath ?deprecated ?replacedBy ?consider ?oldParent ?deprecation_comment WHERE
{
?id rdfs:label ?term .
?id oboInOwl:inSubset ?subsetpath .
OPTIONAL {?id :obsolete_since ?obsolete_since .}
OPTIONAL {?id owl:deprecated ?deprecated .}
OPTIONAL {?id oboInOwl:replacedBy ?replacedBy .}
OPTIONAL {?id oboInOwl:consider ?consider .}
OPTIONAL {?id :oldParent ?oldParent .}
OPTIONAL {?id :deprecation_comment ?deprecation_comment .}
}
"""

# Run SPARQL query and collate results
errfound = False
report = list()
results = g.query(query_term)

Analyse results of query.

In [5]:
report.append('Invalid properties found for these non-obsolete concepts:')

for r in results :
#    print(str(r['id']), str(r['term']), str(r['ext']), str(r['exact_syn']))
    id   = str(r['id'])
    term = str(r['term'])
    subsetpath  = str(r['subsetpath'])
    obsolete_since  = str(r['obsolete_since'])
    deprecated = str(r['deprecated'])
    replacedBy = str(r['replacedBy'])
    consider = str(r['consider'])
    oldParent = str(r['oldParent'])
    deprecation_comment = str(r['deprecation_comment'])

    # Check for (and skip) obsolete concepts
    if "obsolete" not in subsetpath:
        if obsolete_since != "None":
            errfound = True
            report.append("Invalid <obsolete_since> element ::: " + id +  ' (' + term + ')')
        if deprecated != "None":
            errfound = True
            report.append("Invalid <owl:deprecated> element ::: " + id +  ' (' + term + ')')
        if replacedBy != "None":
            errfound = True
            report.append("Invalid <oboInOwl:replacedBy> element ::: " + id +  ' (' + term + ')')
        if consider != "None":
            errfound = True
            report.append("Invalid <oboInOwl:consider> element ::: " + id +  ' (' + term + ')')
        if oldParent != "None":
            errfound = True
            report.append("Invalid <oldParent> element ::: " + id +  ' (' + term + ')')
        if deprecation_comment != "None":
            errfound = True
            report.append("Invalid <deprecation_comment> element ::: " + id +  ' (' + term + ')')
        continue

Write report and return approriate value.

In [8]:
import json

output = {}

# Return exit code (raises exception)
if errfound:
    output = {"test_name":"annotationDeprecationMisuse", "status":WARN, "reason": '\n'.join(report)}

else:
    output = {"test_name":"annotationDeprecationMisuse", "status":NOERR, "reason": "-"}

print(json.dumps(output))

{"test_name": "annotationDeprecationMisuse", "status": "WARN", "reason": "Invalid properties found for these non-obsolete concepts:\nInvalid <obsolete_since> element ::: http://edamontology.org/format_3623 (Index format)\nInvalid <owl:deprecated> element ::: http://edamontology.org/format_3623 (Index format)\nInvalid <oboInOwl:replacedBy> element ::: http://edamontology.org/format_3623 (Index format)\nInvalid <oldParent> element ::: http://edamontology.org/format_3623 (Index format)\nInvalid <deprecation_comment> element ::: http://edamontology.org/format_3623 (Index format)\nInvalid <obsolete_since> element ::: http://edamontology.org/format_3623 (Index format)\nInvalid <owl:deprecated> element ::: http://edamontology.org/format_3623 (Index format)\nInvalid <oboInOwl:replacedBy> element ::: http://edamontology.org/format_3623 (Index format)\nInvalid <oldParent> element ::: http://edamontology.org/format_3623 (Index format)\nInvalid <deprecation_comment> element ::: http://edamontology